In [1]:
"""
Created on Friday 12th March 2021
@author: Aditya Asopa, Bhalla Lab, NCBS
"""
## Import libraries
import sys
import os
import importlib
import pathlib
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pathlib import Path


from eidynamics import ephys_classes
from eidynamics import ephys_functions
from eidynamics import abf_to_data
from eidynamics import plot_tools, utils
from eidynamics.errors      import *

%matplotlib widget

In [ ]:
inputABFfile = Path(r"C:\Users\adity\OneDrive\NCBS\Lab\Projects\EI_Dynamics\Data\23-06-06_GrikAA316\2023_06_06_0007_repeat1.abf")
coordpath = Path(r"C:\Users\adity\OneDrive\NCBS\Lab\Projects\EI_Dynamics\Analysis\polygonProtocols\all_sq.txt")
# parse abf files
cell = abf_to_data.abf_to_data(inputABFfile, exclude_channels=[3], baseline_criterion=0.1, sweep_baseline_epoch=[0, 0.15], baseline_subtraction=True, signal_scaling=1, sampling_freq=2e4, filter_type='bessel', filter_cutoff=1000,
                data_order="sweepwise", plot_data=False )


In [2]:
## CC dataframe
ccdf_path = Path(r"C:\Users\adity\OneDrive\NCBS\Lab\Projects\EI_Dynamics\Analysis\parsed_data\all_cells_FreqSweep_combined_short.h5")
data_all = pd.read_hdf(ccdf_path)
# identify all rows whose 'peak_cell' is not NaN
data = data_all[ pd.notnull(data_all['peaks_cell'])]